# Synthetic CPS


Use the [CPS data.](https://github.com/open-source-economics/taxdata/blob/master/cps_data/cps.csv.gz)

## Setup

In [2]:
import pandas as pd
import numpy as np

from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn import linear_model

Random forest and GBM options.

In [3]:
N_ESTIMATORS = 50

### Graph options

### Data

#### Load

In [4]:
raw = pd.read_csv('https://github.com/open-source-economics/taxdata/raw/master/cps_data/cps.csv.gz')

### Preprocess

Take 10% sample and drop identifiers and other fields unnecessary for synthesis ([source](http://open-source-economics.github.io/Tax-Calculator/)):

* `RECID`: Unique numeric identifier for filing unit; appears as RECID variable in tc CLI minimal output
* `h_seq`:  CPS household sequence number (not used in tax-calculation logic)
* `ffpos`: CPS family identifier within household (not used in tax-calculation logic)
* `a_lineno`: CPS line number for the person record of the head of the tax filing unit (not used in tax-calculation)
* `agi_bin`: Historical AGI category used in data extrapolation...not used in tax calculations
* `filer`: 1 if unit files an income tax return; 0 if not (not used in tax-calculation logic); in the puf.csv file a value of 1 indicates record is from IRS/SOI PUF and 0 indicates record is from CPS
* `fips`: FIPS state code (not used in tax-calculation logic)
* `FLPDYR`: Calendar year for which taxes are calculated
* `pencon_s` and `pencon_p`: Zero in CPS.

In [5]:
DROP_VARS = ['RECID', 'h_seq', 'ffpos', 'a_lineno', 'agi_bin', 'filer', 
             'fips', 'FLPDYR', 'pencon_s', 'pencon_p']

In [6]:
SAMPLE_FRAC = 1.0
dat = raw.sample(frac=SAMPLE_FRAC).drop(DROP_VARS, axis=1)

Set variables with under 10 distinct values to string for classification.

In [7]:
dat.nunique().sort_values().head(20)

DSI                     2
blind_head              2
blind_spouse            2
MARS                    3
EIC                     4
elderly_dependents      4
nu05                    5
n1820                   7
mcare_ben               9
n21                     9
f2441                  11
n24                    11
nu13                   11
nu18                   13
XTOT                   14
mcaid_ben              14
age_head               82
age_spouse             82
e00800                126
e03240                548
dtype: int64

In [8]:
var_nunique = dat.T.apply(lambda x: x.nunique(), axis=1)

In [9]:
class_vars = var_nunique[var_nunique < 10].index.tolist()
class_vars

['DSI',
 'blind_head',
 'blind_spouse',
 'n1820',
 'n21',
 'mcare_ben',
 'nu05',
 'elderly_dependents',
 'EIC',
 'MARS']

Split into test and train.

In [10]:
train, test = train_test_split(dat, test_size=0.5, random_state=42)

In [11]:
# Only make train fields strings, since it's just for modeling. 
# Convert back later.
train[class_vars] = train[class_vars].astype(str)

/home/maxghenis/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


## Functions

From https://stackoverflow.com/a/52615768/1840471

In [12]:
def percentile_qarray_np(dat, q):
  return np.apply_along_axis(
    lambda x: np.percentile(x[1:], x[0]),
    1,
    np.concatenate([np.array(q)[:, np.newaxis], dat], axis=1)
  )

In [13]:
def rf_quantile(m, X, q):
    rf_preds = []
    for estimator in m.estimators_:
        rf_preds.append(estimator.predict(X))
    rf_preds = np.array(rf_preds).transpose()  # One row per record.
    return percentile_qarray_np(rf_preds, q * 100)

## Synthesize

Seed variables should be those whose combination would be freely published by the IRS.

In [14]:
SEED_COLS = ['MARS', 'XTOT', 'age_head']

Seed the synthetic dataset with a 100% sample (with replacement) of the training set's seed columns. Use `copy` to avoid a warning.

In [15]:
synth = train.copy()[SEED_COLS].sample(frac=1.0, replace=True, random_state=42)

In [16]:
rf = ensemble.RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                    min_samples_leaf=1, random_state=3, 
                                    verbose=True, 
                                    n_jobs=-1)  # Use maximum number of cores.

### Set order of feature synthesis

Start with demographic columns, then income, then benefits, then weight.

In [17]:
# MARS and XTOT would also go here if not the seed columns.
DEMO_COLS = ['MARS', 'XTOT', 'age_head', 
             'age_spouse', 'blind_head', 'blind_spouse', 'DSI',
             'nu05', 'nu13', 'n24', 'nu18', 'n1820', 'n21',
             'elderly_dependents',
             'f2441', 'EIC']
demo_cols = list(set(DEMO_COLS) - set(SEED_COLS))
ben_cols = [x for x in test.columns if x.endswith('_ben')]
income_cols = list(set(test.columns) - set(demo_cols) - set(ben_cols) - 
                   set(['s006']) - set(SEED_COLS))

In [18]:
rf_vars = demo_cols + income_cols + ben_cols + ['s006']

In [19]:
%%time
for i, col in enumerate(rf_vars):
  print('Synthesizing feature ' + str(i + 1) + ' of ' + 
        str(len(rf_vars)) + ': ' + col + '...')
  rf.fit(train[synth.columns], train[col])
  synth[col] = rf_quantile(rf, synth, np.random.rand(synth.shape[0]))

Synthesizing feature 1 of 56: age_spouse...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    2.7s finished


Synthesizing feature 2 of 56: DSI...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    4.1s finished


Synthesizing feature 3 of 56: n21...


...

Synthesizing feature 55 of 56: other_ben...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.0min finished


Synthesizing feature 56 of 56: s006...


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  4.5min finished


CPU times: user 4h 45min 8s, sys: 6min 10s, total: 4h 51min 18s
Wall time: 12h 8min


## Evaluate

Compare CDFs and correlations to test data.

In [20]:
synthr = synth.round()

In [21]:
def cdf(df, col):
  res = df[[col, 's006']].sort_values(col)
  res['s006_cumpct'] = res.s006.cumsum() / res.s006.sum()
  return res

In [22]:
from matplotlib.ticker import MaxNLocator

def compare_cdf(synth, test, col, unit_prepend=''):
  synth_cdf = cdf(synth, col)
  test_cdf = cdf(test, col)
  ax = synth_cdf.plot(x=col, y='s006_cumpct')
  test_cdf.plot(x=col, y='s006_cumpct', ax=ax, color='#BDBDBD')
  ax.legend(['Synthetic', 'Test'])
  ax.grid(color='#eeeeee')
  ax.xaxis.set_major_locator(MaxNLocator(integer=True))
  # Assume a dollar amount if exceeds 100. Use symlog and $ axis.
  if test[col].max() > 100:
    unit_prepend = '$'
    plt.xscale('symlog')
  ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(
      lambda x, _: unit_prepend + format(int(x), ',')))
  ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(
      lambda y, _: '{:.0%}'.format(y)))
  ax.set(xlabel=col, ylabel='Share of tax units')
  plt.title('CDF of ' + col + ' for synthetic and test sets', loc='left')
  sns.despine(left=True, bottom=True)
  plt.show()

In [23]:
# for i in rf_vars:
#     if i != 's006':
#         compare_cdf(synthr, test, i)

In [24]:
synth.to_csv('synth.csv')

In [25]:
test.to_csv('test.csv')

In [26]:
train.to_csv('train.csv')

## synthpop

In [27]:
# Also requires installing tzlocal
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: During startup - 
  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Warning message:

  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: package 'RevoUtils' was built under R version 3.4.3 

  warnings.warn(x, RRuntimeWarning)


In [29]:
robjects.r('library(synthpop)')

/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: lattice

  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: MASS

  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: nnet

  warnings.warn(x, RRuntimeWarning)
/home/maxghenis/miniconda3/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required package: ggplot2

  warnings.warn(x, RRuntimeWarning)


array(['synthpop', 'ggplot2', 'nnet', 'MASS', 'lattice', 'tools',
       'RevoUtils', 'stats', 'graphics', 'grDevices', 'utils', 'datasets',
       'RevoUtilsMath', 'methods', 'base'], dtype='<U13')

In [30]:
robjects.globalenv['train'] = train

In [31]:
%%time
synth_synthpop = robjects.r('syn(train)$syn')


Variable(s):
 
DSI, blind_head, blind_spouse, n1820, n21, mcare_ben, nu05, elderly_dependents, EIC, MARS
 
have been changed from character to factor.


Variables 

nu13, f2441

 are collinear.


Variables later in visit.sequence are derived from 

nu13

.


Variables 

e19800, e20100

 are collinear.


Variables later in visit.sequence are derived from 

e19800

.

syn  variables



1

   
 

age_head
 

age_spouse
 

e00200p
 

e00900p
 

e02100p
 

e00200s
 

e00900s
 

e02100s
 

e00600
 

e00800

    
 

e01500
 

e02300
 

s006
 

DSI
 

blind_head
 

blind_spouse
 

nu18
 

n1820
 

n21
 

ssi_ben

    
 

vet_ben
 

snap_ben
 

mcare_ben
 

mcaid_ben
 

e02400
 

tanf_ben
 

housing_ben
 

wic_ben
 

nu13
 

nu05

    
 

n24
 

elderly_dependents
 

f2441
 

EIC
 

XTOT
 

MARS
 

e01100
 

e01400
 

e03300
 

e03270

    
 

e20400
 

e32800
 

e19200
 

e18500
 

e03240
 

e17500
 

e18400
 

e00900
 

e00650
 

e00300

    
 

e00400
 

e01700
 

e19800
 

e20100
 

e03210

In [32]:
synth_synthpop.iloc[0]

age_head                  42
age_spouse                 0
e00200p                    0
e00900p                    0
e02100p                    0
e00200s                    0
e00900s                    0
e02100s                    0
e00600                     0
e00800                     0
e01500                     0
e02300                     0
s006                  119900
DSI                        0
blind_head                 0
blind_spouse               0
nu18                       0
n1820                      0
n21                        1
ssi_ben                12685
vet_ben                    0
snap_ben                3668
mcare_ben                  0
mcaid_ben               7031
e02400                     0
tanf_ben                   0
housing_ben             8677
wic_ben                    0
nu13                       0
nu05                       0
n24                        0
elderly_dependents         0
f2441                      0
EIC                        0
XTOT          

In [33]:
synth_synthpop.head()

,age_head,age_spouse,e00200p,e00900p,e02100p,e00200s,e00900s,e02100s,e00600,e00800,...,e00300,e00400,e01700,e19800,e20100,e03210,e03150,other_ben,e00200,e02100
387379,42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,55.0,0.0,0.0,0.0,0,0.0,0.0,8589.0,0.0,0.0
281216,72,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
404626,74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,542.0,0.0,0.0,0.0,0,0.0,0.0,9948.0,0.0,0.0
220030,36,36.0,85392.0,0.0,0.0,108090.0,0.0,0.0,0.0,0.0,...,2743.0,0.0,0.0,0.0,0,99.0,0.0,0.0,186925.0,0.0
240671,66,0.0,19892.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,5744.0,35520.0,0.0


In [34]:
synth_synthpop.to_csv('synth_synthpop.csv')

In [35]:
synth_synthpop.dtypes

age_head                int32
age_spouse            float64
e00200p               float64
e00900p               float64
e02100p               float64
e00200s               float64
e00900s               float64
e02100s               float64
e00600                float64
e00800                float64
e01500                float64
e02300                float64
s006                  float64
DSI                    object
blind_head             object
blind_spouse           object
nu18                  float64
n1820                  object
n21                    object
ssi_ben               float64
vet_ben               float64
snap_ben              float64
mcare_ben              object
mcaid_ben             float64
e02400                float64
tanf_ben              float64
housing_ben           float64
wic_ben               float64
nu13                  float64
nu05                   object
n24                   float64
elderly_dependents     object
f2441                   int32
EIC       